In [61]:
import pandapower as pp
import pandapower.networks as pn
from pandapower.estimation import estimate, remove_bad_data, chi2_analysis
import pandas as pd
import numpy as np
# from tqdm import tqdm
print(pp.__version__)
import matplotlib.pyplot as plt
import ruptures as rpt
from tqdm import tqdm
# from julia.PowerModels import run_ac_opf
# from julia.PowerModels import run_ac_opf

2.0.1


In [62]:
def create_measurement_unit(df_measurement, net):
    list_value=[]
    list_std=[]
    for index, row in df_measurement.iterrows():
        if row['element_type'] =='bus':
            if row['meas_type'] =='v':
                mu= net.res_bus.iloc[row['element'],0]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
            elif row['meas_type'] =='p':
                mu= net.res_bus.iloc[row['element'],2]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
            elif row['meas_type'] =='q':
                mu= net.res_bus.iloc[row['element'],3]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
                
        elif row['element_type'] =='line':
            if row['side']=='from':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],0]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],1]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],6]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
            elif row['side']=='to':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],2]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],3]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],7]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                    
                    
        elif row['element_type'] =='trafo':
            if row['side']=='from':
                if row['meas_type'] == 'i':
                    mu =net.res_trafo.iloc[row['element'],6]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] == 'p':
                    mu =net.res_trafo.iloc[row['element'],0]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] == 'q':
                    mu =net.res_trafo.iloc[row['element'],1]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
            elif row['side']=='to':
                if row['meas_type'] =='i':
                    mu =net.res_trafo.iloc[row['element'],7]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] =='p':
                    mu =net.res_trafo.iloc[row['element'],2]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] =='q':
                    mu =net.res_trafo.iloc[row['element'],3]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
#         print(mu)  
        value = np.random.normal(mu, sigma, 1)
        list_value.append(value[0])
        list_std.append(sigma)
    df_measurement['value']=list_value
    df_measurement['std_dev']=list_std

    for index, row in df_measurement.iterrows():
        if row['element_type']=='bus':
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'])
        elif row['element_type']=='line':
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'], side=row['side'])
        elif row['element_type']=='trafo':
            if row['meas_type'] in ['p','q']:
                pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                      std_dev=row['std_dev'], element=row['element'], side=row['side'])
    return df_measurement, net

In [63]:
global upper_bus_accuracy, lower_bus_accuracy, upper_line_accuracy, lower_line_accuracy, upper_trafo_accuracy, lower_trafo_accuracy
upper_bus_accuracy=1.01
lower_bus_accuracy=0.99
upper_line_accuracy=1.03
lower_line_accuracy=0.97
upper_trafo_accuracy=1.03
lower_trafo_accuracy=0.97

In [64]:
net = pn.create_cigre_network_mv(with_der="pv_wind")
net.bus=net.bus.sort_index()
net.line=net.line.sort_index()
net.trafo=net.trafo.sort_index()
net.load=net.load.sort_index()
net.sgen=net.sgen.sort_index()
net.sgen.p_mw=np.random.randint(50, size=len(net.sgen))/1000
net.sgen.sn_mva=net.sgen.p_mw
# net.shunt=net.shunt.sort_index()
net.switch.closed=[True]*2+[True]*2+[True]*2+[True]*2

zero_inject_bus= list(set(net.bus.index).difference(set(np.where(net.gen.p_mw!=0)[0]).union(set(net.load.bus)).union(net.ext_grid.bus).union(net.shunt.bus)))
list_bus_meas=list(set(net.bus.index)-set(zero_inject_bus))
list_line_meas=list(set(net.line.index))
list_transfo_meas=list(set(net.trafo.index))
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v']*len(list_bus_meas)+['p','q','p','q']*len(list_line_meas)+['p','q','p','q']*len(list_transfo_meas)
df_measurement['element_type']=['bus']*len(list_bus_meas)+['line','line','line','line']*len(list_line_meas)+['trafo','trafo','trafo','trafo']*len(list_transfo_meas)
df_measurement['element']=[item for item in list_bus_meas]+[item for item in list_line_meas for x in range(4)]+[item for item in list_transfo_meas for x in range(4)]
df_measurement['side']=['None']*len(list_bus_meas)+['from','from','to','to']*len(list_line_meas)+['from','from','to','to']*len(list_transfo_meas)


pp.runpp(net)
net.res_bus=net.res_bus.sort_index()
net.res_line=net.res_line.sort_index()
net.res_trafo=net.res_trafo.sort_index()
df_measurement, net = create_measurement_unit(df_measurement, net)
success = estimate(net, init="slack", zero_injection=zero_inject_bus, calculate_voltage_angles=True)

In [65]:
net.measurement.head()

,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,0,1.034450,0.005150,None
1,None,v,bus,1,0.995040,0.004980,None
2,None,v,bus,3,0.961236,0.004809,None
3,None,v,bus,4,0.961745,0.004800,None
4,None,v,bus,5,0.949843,0.004796,None


In [66]:
net.measurement.iloc[1,4]=0.9
print(net.measurement.head())

   name measurement_type element_type  element     value   std_dev  side
0  None                v          bus        0  1.034450  0.005150  None
1  None                v          bus        1  0.900000  0.004980  None
2  None                v          bus        3  0.961236  0.004809  None
3  None                v          bus        4  0.961745  0.004800  None
4  None                v          bus        5  0.949843  0.004796  None


In [69]:
success_chi2 = chi2_analysis(net, init="slack")
print('Chi square test:', success_chi2)

Chi square test: True


In [43]:
net.measurement.head()

,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,0,1.023254,0.005150,None
1,None,v,bus,1,0.900000,0.004980,None
2,None,v,bus,3,0.964891,0.004813,None
3,None,v,bus,4,0.964986,0.004804,None
4,None,v,bus,5,0.954120,0.004800,None


In [45]:
success_rn_max = remove_bad_data(net, init="slack", calculate_voltage_angles=True)
print('remove data: ', success_rn_max)

remove data:  True


In [57]:
print(chi2_analysis)

<function chi2_analysis at 0x10db2d6a8>


In [46]:
net.measurement.head()

,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,0,1.023254,0.005150,None
2,None,v,bus,3,0.964891,0.004813,None
3,None,v,bus,4,0.964986,0.004804,None
4,None,v,bus,5,0.954120,0.004800,None
5,None,v,bus,6,0.955119,0.004799,None


In [48]:
list(net)

['bus',
 'load',
 'sgen',
 'storage',
 'gen',
 'switch',
 'shunt',
 'ext_grid',
 'line',
 'trafo',
 'trafo3w',
 'impedance',
 'dcline',
 'ward',
 'xward',
 'measurement',
 'pwl_cost',
 'poly_cost',
 'line_geodata',
 'bus_geodata',
 '_empty_res_bus',
 '_empty_res_ext_grid',
 '_empty_res_line',
 '_empty_res_trafo',
 '_empty_res_trafo3w',
 '_empty_res_load',
 '_empty_res_sgen',
 '_empty_res_storage',
 '_empty_res_gen',
 '_empty_res_shunt',
 '_empty_res_impedance',
 '_empty_res_dcline',
 '_empty_res_ward',
 '_empty_res_xward',
 '_ppc',
 '_is_elements',
 '_pd2ppc_lookups',
 'version',
 'converged',
 'name',
 'f_hz',
 'sn_mva',
 'std_types',
 'res_bus',
 'res_line',
 'res_trafo',
 'res_trafo3w',
 'res_impedance',
 'res_ext_grid',
 'res_load',
 'res_sgen',
 'res_storage',
 'res_shunt',
 'res_gen',
 'res_ward',
 'res_xward',
 'res_dcline',
 'user_pf_options',
 '_options',
 'OPF_converged',
 '_impedance_bb_switches',
 '_isolated_buses',
 '_gen_order',
 'res_bus_power_flow',
 'res_ext_grid_power

In [54]:
net._gen_order

{'ext_grid': (0, 1)}

In [59]:
net.bad_data_present()

AttributeError: 'pandapowerNet' instance has no attribute 'bad_data_present'